#  Data analysis for Lexical Richness

In [330]:
#importing libraries
import os
import nltk
import re
import numpy as np
import regex
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
import pandas as pd

# Functions

In [331]:
#binary search for strings
def binarySearchArr(arr, x):
    l = 0
    r = len(arr)
    while (l <= r):
        m = l + ((r - l) // 2)
 
        res = (x == arr[m])
        # Check if x is present at mid
        if (res == True):
            return m
 
        # If x greater, ignore left half
    #problem is here
        elif (res == False):
            if (x>arr[m]):
                l = m+1
            elif (x<arr[m]):
                r=m-1
        # If x is smaller, ignore right half
    return -1

In [332]:
#This function removes all of the rows that contain a string in the passed column
def rmStr (df, col):
    row_index=0
    row_ind=[]
    for i in df[col]:
        try:
            int(i)
        except:
            row_ind.append(row_index)
        row_index+=1
    return df.drop(labels=row_ind,axis=0)

In [333]:
# function that finds the unique values in a list
def get_uniques(x):
    return list(x.unique())

In [334]:
#finds the number of texts per language in the given list of anon_ids
def find_counts(df,anon_list,lang_col):
    spanish=0
    chinese=0
    korean=0
    index=0
    for i in df["Number of Questions"]:
        anon_id=str(df.at[index,'anon_id'])
        if anon_id in anon_list:
            if df.at[index,lang_col]=='Spanish':            
                spanish+=i
            elif df.at[index,lang_col]=='Chinese':
                chinese+=i
            elif df.at[index,lang_col]=='Korean':
                korean+=i
        index+=1
    return(spanish,chinese,korean)

In [335]:
# reference code from previous research, tokinizes essays
# NEEDS UPDATING TO FIT CSV FILES

#function that does this
def calculate_tokens(text):
    essay = text

        
    tokenizer = RegexpTokenizer(r'[0-9a-zA-Z_\-]+')
    tokenized = tokenizer.tokenize(essay)
    unique = np.unique(tokenized)
    
    return {
        'total': len(tokenized),
        'unique': len(unique)
    }


In [336]:
# function that counts the amount of tokens per language FINDS AVERAGE OF ALL LVLS IN SPECIFIED LIST
def finding_tokens(df,id_list,lang_col,text_len_col):
    spanish=0
    korean=0
    chinese=0
    index=0
    for i in df['anon_id']:
        if i in id_list:
            lang=df.at[index,lang_col]
            length=int(df.at[index,text_len_col])
            if lang=='Spanish':
                spanish+=length
            elif lang=='Korean':
                korean+=length
            elif lang=='Chinese':
                chinese+=length
        index+=1
    print('Spanish Token Average: ',spanish/lspanish,'\nKorean Token Average: ',korean/lkorean,
          '\nChinese Token Average:',chinese/lchinese)

In [337]:
# function that counts the amount of tokens per language FINDS AVERAGE OF SPECIFIED LEVEL IN LIST
def finding_tokens_lvl(df,id_list,lang_col,text_len_col,lvl):
    spanish=0
    korean=0
    chinese=0
    index=0
    for i in df['anon_id']:
        if i in id_list:
            lang=df.at[index,lang_col]
            length=int(df.at[index,text_len_col])
            if (df.at[index,'level_id']==lvl):
                if lang=='Spanish':
                    spanish+=length
                elif lang=='Korean':
                    korean+=length
                elif lang=='Chinese':
                    chinese+=length
        index+=1
    print('Spanish Token Average: ',spanish/lspanish,'\nKorean Token Average: ',korean/lkorean,
          '\nChinese Token Average:',chinese/lchinese)

# Adding DFs

In [338]:
loose=pd.read_csv('original-sheets\Responses-Loose.csv')
student_info=pd.read_csv('corpus-files\student_information.csv')
course=pd.read_csv('corpus-files\course.csv')
answer_df=pd.read_csv('answer.csv',index_col = 'answer_id', low_memory=False)

In [339]:
course_=course.drop(columns=['class_id','semester','section'])

student=student_info.drop(columns=['birth_year','language_used_at_home','non_native_language_1','yrs_of_study_lang1',
                                  'study_in_classroom_lang1','ways_of_study_lang1','non_native_language_2','study_in_classroom_lang2',
                                  'ways_of_study_lang2','non_native_language_3','yrs_of_study_lang3','study_in_classroom_lang3',
                                  'ways_of_study_lang3','yrs_of_english_learning','yrs_in_english_environment','yrs_of_study_lang2'])
answer_=answer_df[['question_id','anon_id','course_id','version','text_len','text','tokens']].reset_index()

In [340]:
loose.head()

,3Langs,Item,answer_id,anon_id,L1,gender,course_id,level_id,class_id,question_id,version,text1_len,text2_len,text3_len,text1,"text2 (line breaks/extra spaces removed, spaces added to reach 60)",text3 (edits made to fix word counts),Judgement,Notes
0,Y,C12,141.0,aj8,Chinese,Male,118.0,5.0,w,17.0,1.0,121.0,114.0,114.0,"\n\n\n Today, I am going to describe one of my...","Today, I am going to describe one of my classm...","Today, I am going to describe one of my classm...",1.0,NaN
1,Y,C13,143.0,az8,Chinese,Female,118.0,5.0,w,17.0,1.0,96.0,95.0,95.0,My niece is 3 years old who is my younger brot...,My niece is 3 years old who is my younger brot...,My niece is 3 years old who is my younger brot...,1.0,NaN
2,Y,K9,133.0,az2,Korean,Male,118.0,5.0,w,17.0,1.0,130.0,128.0,128.0,"When I was in Germany, I met a friend who was ...","When I was in Germany, I met a friend who was ...","When I was in Germany, I met a friend who was ...",1.0,NaN
3,Y,K11,135.0,at8,Korean,Female,118.0,5.0,w,17.0,3.0,104.0,105.0,105.0,my friend is ANON_NAME_0. she is a my ELI frie...,my friend is ANON_NAME_0. she is a my ELI frie...,my friend is ANON_NAME_0. she is a my ELI frie...,1.0,NaN
4,Y,K20,188.0,eh9,Korean,Male,118.0,5.0,w,17.0,1.0,98.0,97.0,97.0,Younghun is my best friend. His facial appeara...,Younghun is my best friend. His facial appeara...,Younghun is my best friend. His facial appeara...,1.0,NaN


In [341]:
#Cleaning the answer df
col='course_id'
answer=rmStr(answer_, col) #removes strings from answer_.course_id 

In [342]:
# Merging the datasets

merge_ans=answer[['anon_id','course_id']] #sets answer df up for merging 
student_merge=student.drop(columns=['course_history']) #set student df up for merging

student_ans=student_merge.merge(answer,on='anon_id').astype({'course_id':'int64'}) #merges student and answers

stu_ans_crs_=student_ans.merge(course_, on='course_id') #merges the student-answers df with course df

In [343]:
stu_ans_crs_

,anon_id,gender,native_language,age,answer_id,question_id,course_id,version,text_len,text,tokens,level_id
0,do6,Female,Russian,31.0,150,4,117,1,299,Some people prefer eat out and some like doing...,"['Some', 'people', 'prefer', 'eat', 'out', 'an...",5
1,do6,Female,Russian,31.0,1221,97,117,1,288,My opinion is that a person does need educatio...,"['My', 'opinion', 'is', 'that', 'a', 'person',...",5
2,do6,Female,Russian,31.0,1957,189,117,1,321,There are two national rooms in the Cathedral ...,"['There', 'are', 'two', 'national', 'rooms', '...",5
3,do6,Female,Russian,31.0,2164,190,117,1,464,There are two nation rooms in the Cathedral of...,"['There', 'are', 'two', 'nation', 'rooms', 'in...",5
4,bv5,Male,Arabic,21.0,151,4,117,1,315,"""Not all learning takes place in the classroom...","['``', 'Not', 'all', 'learning', 'takes', 'pla...",5
...,...,...,...,...,...,...,...,...,...,...,...,...
46225,cy7,Female,Korean,NaN,47682,6066,1034,1,10,1. emphasis\n2. appropriate\n3. requires\n4. a...,"['1', '.', 'emphasis', '2', '.', 'appropriate'...",3
46226,fp7,Female,Turkish,NaN,47823,6066,1034,1,10,1.emphasis \n2.appropriate\n3.requires\n4.anal...,"['1', '.', 'emphasis', '2', '.', 'appropriate'...",3
46227,fq6,Male,Chinese,NaN,47824,6066,1034,1,10,1.emphasis\n2.appropriate\n3.requires\n4.analy...,"['1', '.', 'emphasis', '2', '.', 'appropriate'...",3
46228,di6,Male,Chinese,NaN,47787,6066,1034,1,10,1 emphasis\n2 normal\n3 requires\n4 analyze\n5...,"['1', 'emphasis', '2', 'normal', '3', 'require...",3


# Cleaning Answers in Whole Dataset

In [344]:
#dropping all rows where the answer is less than 60 words
index=0
ind_list=[]
for i in stu_ans_crs_.text_len:
    txtlen=int(i)
    if txtlen < 60:
        ind_list.append(index)
    index+=1

stu_ans_crs_=stu_ans_crs_.drop(labels=ind_list,axis=0).reset_index()

In [380]:
# adding spaces after punctuation if needed
index=0
stu_ans_crs=stu_ans_crs_.copy()
for text in testing.text:    
    stu_ans_crs.at[index,'text']=re.sub(r'(?<=[.,!?)\n])(?=[^\s])', r' ', text)
    index+=1

In [373]:
print('Before adding spaces:\n',stu_ans_crs_.at[690,'text'],'\n')
print('After adding spaces:\n',stu_ans_crs.at[690,'text'])

Before adding spaces:
 In my country we usually don't use tea bags. First we boil the water,then we put some of the hot water at the top.We add the tea at the top. We wait like 20 minutes with low heat for it to be ready.At last, we first poor the water with tea,then we add some pure hot water in it. 

After adding spaces:
 In my country we usually don't use tea bags. First we boil the water, then we put some of the hot water at the top. We add the tea at the top. We wait like 20 minutes with low heat for it to be ready. At last, we first poor the water with tea, then we add some pure hot water in it.


In [347]:
word_tokenize(stu_ans_crs.at[690,'text'])

['In',
 'my',
 'country',
 'we',
 'usually',
 'do',
 "n't",
 'use',
 'tea',
 'bags',
 '.',
 'First',
 'we',
 'boil',
 'the',
 'water',
 ',',
 'then',
 'we',
 'put',
 'some',
 'of',
 'the',
 'hot',
 'water',
 'at',
 'the',
 'top',
 '.',
 'We',
 'add',
 'the',
 'tea',
 'at',
 'the',
 'top',
 '.',
 'We',
 'wait',
 'like',
 '20',
 'minutes',
 'with',
 'low',
 'heat',
 'for',
 'it',
 'to',
 'be',
 'ready',
 '.',
 'At',
 'last',
 ',',
 'we',
 'first',
 'poor',
 'the',
 'water',
 'with',
 'tea',
 ',',
 'then',
 'we',
 'add',
 'some',
 'pure',
 'hot',
 'water',
 'in',
 'it',
 '.']

In [430]:
index=0
for text in stu_ans_crs.text:   
    words=text.strip().split(" ")  #creates a list of each word in the text by seperating them whenever there is a space
    for word in words:
        if word=='\n': #sometimes it counted \n as a word, so this just gets rid of instances of that if it happens
            words.remove(word)
    stu_ans_crs.at[index,'text_len']=len(words) #changes the text_len column to include the fixed word counts
    index+=1
stu_ans_crs

,index,anon_id,gender,native_language,age,answer_id,question_id,course_id,version,text_len,text,tokens,level_id
0,0,do6,Female,Russian,31.0,150,4,117,1,299,Some people prefer eat out and some like doing...,"['Some', 'people', 'prefer', 'eat', 'out', 'an...",5
1,1,do6,Female,Russian,31.0,1221,97,117,1,288,My opinion is that a person does need educatio...,"['My', 'opinion', 'is', 'that', 'a', 'person',...",5
2,2,do6,Female,Russian,31.0,1957,189,117,1,317,There are two national rooms in the Cathedral ...,"['There', 'are', 'two', 'national', 'rooms', '...",5
3,3,do6,Female,Russian,31.0,2164,190,117,1,459,There are two nation rooms in the Cathedral of...,"['There', 'are', 'two', 'nation', 'rooms', 'in...",5
4,4,bv5,Male,Arabic,21.0,151,4,117,1,308,"""Not all learning takes place in the classroom...","['``', 'Not', 'all', 'learning', 'takes', 'pla...",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17252,46214,bh8,Male,Japanese,NaN,48034,6087,1033,1,107,I received my Medical Doctor license in Japan ...,"['I', 'received', 'my', 'Medical', 'Doctor', '...",3
17253,46215,bh8,Male,Japanese,NaN,48293,6119,1033,1,138,I introduce my ideal home to you quickly. This...,"['I', 'introduce', 'my', 'ideal', 'home', 'to'...",3
17254,46217,cz3,Female,Korean,NaN,47451,6027,1033,1,102,I will put 5 items in time capsule such as som...,"['I', 'will', 'put', '5', 'items', 'in', 'time...",3
17255,46218,cz3,Female,Korean,NaN,48001,6087,1033,1,104,I have studied Visual Design since 2002. I gra...,"['I', 'have', 'studied', 'Visual', 'Design', '...",3


# Getting Ids of participants in all three of Whole Dataset

In [431]:
sac_lvls=stu_ans_crs[['anon_id','native_language','level_id']]
sac_lvls_rkc=sac_lvls

index=0
for i in sac_lvls_rkc.native_language:
    if (i != 'Korean' and i != 'Spanish' and i != 'Chinese'):
        sac_lvls_rkc=sac_lvls_rkc.drop(index)
    index+=1

In [432]:
sac_lvls_rkc

,anon_id,native_language,level_id
13,ax4,Chinese,5
14,ax4,Chinese,5
15,ax4,Chinese,5
16,ax4,Chinese,5
17,ax4,Chinese,5
...,...,...,...
17243,ew6,Chinese,3
17244,ew6,Chinese,3
17254,cz3,Korean,3
17255,cz3,Korean,3


In [433]:
# allows us to see what levels each anon_id participated in 
lvl_list=sac_lvls_rkc.groupby('anon_id').agg(n_uniq=('level_id','nunique'), lvl_nums=('level_id',get_uniques))
lvl_list=lvl_list.reset_index()

In [434]:
lvl_list

,anon_id,n_uniq,lvl_nums
0,aa0,1,[5]
1,aa1,1,[4]
2,aa3,1,[4]
3,aa8,2,"[5, 4]"
4,aa9,3,"[3, 4, 2]"
...,...,...,...
471,ha0,1,[5]
472,ha2,2,"[5, 4]"
473,ha6,2,"[3, 4]"
474,hb4,3,"[5, 4, 3]"


In [438]:
# will create a list of only the ids that have levels 3, 4, and 5 in their lvl_nums column
all_three=[]
ind=0
for i in lvl_list.anon_id:
    if ((3 in lvl_list.iat[ind,2]) and (4 in lvl_list.iat[ind,2]) and (5 in lvl_list.iat[ind,2])):
        append=lvl_list.at[ind, 'anon_id']
        all_three.append(append)
    ind+=1

all_three.sort()
len(all_three)

42

In [439]:
all_anon=[]
for i in sac_lvls_rkc.anon_id:
    all_anon.append(i)
len(all_anon)
all_anon.sort()

sorted_sac=sac_lvls_rkc.sort_values(by=['anon_id']).reset_index(drop=True)

In [440]:
lang_dict_all={}
for i in all_three:
    df_ind=binarySearchArr(all_anon,i)
    key=sorted_sac.at[df_ind,'native_language']
    val=sorted_sac.at[df_ind,'anon_id']
    
    if key in lang_dict_all:
        lang_dict_all[key].append(val)
    else:
        lang_dict_all[key]=[val]
    
print(lang_dict_all)

{'Korean': ['ag9', 'an5', 'as0', 'ay1', 'be7', 'bv9', 'bz5', 'cc4', 'cj8', 'co5', 'cv3', 'cw0', 'ea4', 'eq8', 'es9', 'fj7', 'fp5', 'fu6', 'gc2', 'gq8', 'gz2', 'hb4'], 'Chinese': ['aq1', 'ar8', 'bf7', 'bl4', 'bl7', 'bp4', 'bz2', 'cb3', 'cf9', 'cz4', 'dk6', 'ev9', 'fi1', 'fj4', 'gb4', 'gx5'], 'Spanish': ['bj2', 'cm9', 'fa2', 'fy1']}


In [441]:
# will create a list of only the ids that have levels 3 and 4 in their lvl_nums column
lvls34=[]
ind=0
for i in lvl_list.anon_id:
    if ((3 in lvl_list.iat[ind,2]) and (4 in lvl_list.iat[ind,2])):
        append=lvl_list.at[ind, 'anon_id']
        lvls34.append(append)
    ind+=1

lvls34.sort()
len(lvls34)

111

In [443]:
lang_dict_34={}
for i in lvls34:
    df_ind=binarySearchArr(all_anon,i)
    key=sorted_sac.at[df_ind,'native_language']
    val=sorted_sac.at[df_ind,'anon_id']
    
    if key in lang_dict_34:
        lang_dict_34[key].append(val)
    else:
        lang_dict_34[key]=[val]
    
print(lang_dict_34)

{'Korean': ['aa9', 'af1', 'ag9', 'ah9', 'an4', 'an5', 'as0', 'as4', 'ay1', 'bc0', 'be2', 'be7', 'bf1', 'bm0', 'br5', 'bv9', 'bz5', 'ca6', 'cc4', 'cf6', 'ch9', 'cj8', 'cn3', 'co5', 'cv3', 'cw0', 'cw6', 'cy5', 'dd1', 'dj6', 'dp5', 'ea4', 'ef4', 'eo2', 'eq8', 'es9', 'et1', 'ex0', 'fc9', 'fh7', 'fi4', 'fj7', 'fk1', 'fl4', 'fl5', 'fo3', 'fp5', 'fp9', 'fu6', 'fv7', 'fv9', 'gb7', 'gc2', 'gd1', 'gl4', 'gq8', 'gv1', 'gz2', 'ha6', 'hb4'], 'Chinese': ['ag6', 'ai4', 'ap5', 'ap8', 'aq1', 'ar8', 'ax7', 'ba3', 'bf7', 'bl4', 'bl7', 'bp4', 'bu9', 'bz2', 'cb3', 'cf9', 'cl2', 'cl6', 'cq2', 'cs5', 'cz4', 'dk6', 'dl8', 'dm4', 'do7', 'dp1', 'dt4', 'dx1', 'eb6', 'ev5', 'ev9', 'ey8', 'ff6', 'fi1', 'fj4', 'fm2', 'fn8', 'fy3', 'gb4', 'gw5', 'gw8', 'gx5'], 'Spanish': ['bj2', 'ch0', 'cm9', 'en3', 'fa2', 'fe7', 'fg7', 'fy1', 'fy6']}


In [444]:
# will create a list of only the ids that have levels 3 and 4 in their lvl_nums column
lvls45=[]
ind=0
for i in lvl_list.anon_id:
    if ((4 in lvl_list.iat[ind,2]) and (5 in lvl_list.iat[ind,2])):
        append=lvl_list.at[ind, 'anon_id']
        lvls45.append(append)
    ind+=1

lvls45.sort()
len(lvls45)

154

In [445]:
lang_dict_45={}
for i in lvls45:
    df_ind=binarySearchArr(all_anon,i)
    key=sorted_sac.at[df_ind,'native_language']
    val=sorted_sac.at[df_ind,'anon_id']
    
    if key in lang_dict_45:
        lang_dict_45[key].append(val)
    else:
        lang_dict_45[key]=[val]
    
print(lang_dict_45)

{'Korean': ['aa8', 'ac5', 'ad1', 'ag9', 'al5', 'an5', 'ap4', 'aq5', 'as0', 'as7', 'au5', 'au6', 'aw6', 'ay1', 'be7', 'bq0', 'bu4', 'bv9', 'bw3', 'bz5', 'cc4', 'ce3', 'ch2', 'ci0', 'cj8', 'co5', 'cp0', 'cv3', 'cw0', 'cw1', 'dd9', 'df3', 'dm3', 'dy7', 'ea4', 'eb3', 'eb9', 'ec1', 'eg5', 'eq8', 'es0', 'es9', 'et3', 'ex3', 'fa0', 'fb4', 'fd6', 'ff1', 'fi5', 'fj7', 'fl0', 'fp5', 'fs0', 'ft2', 'fu6', 'fx4', 'fz8', 'ga1', 'gc2', 'ge6', 'gg2', 'gg6', 'gj0', 'gk5', 'gl1', 'gn0', 'gq8', 'gs3', 'gu0', 'gz2', 'ha2', 'hb4'], 'Chinese': ['ad7', 'ae9', 'af4', 'am5', 'an7', 'aq1', 'ar8', 'ar9', 'az8', 'bd7', 'bf2', 'bf7', 'bf9', 'bl4', 'bl7', 'bp2', 'bp4', 'br9', 'bv8', 'bw9', 'by5', 'by6', 'bz1', 'bz2', 'ca4', 'cb3', 'cd6', 'cf9', 'ci2', 'cj5', 'cl3', 'cz2', 'cz4', 'dc0', 'di7', 'dk6', 'dm8', 'dq9', 'du9', 'dw2', 'ei2', 'ei8', 'eo1', 'eq4', 'es4', 'et4', 'ev6', 'ev9', 'ff2', 'fg8', 'fi1', 'fj4', 'fk8', 'fo4', 'fq5', 'fw7', 'ga3', 'gb4', 'gf5', 'gl2', 'gm1', 'go8', 'gt0', 'gv3', 'gw0', 'gx5'], 'Spanish

In [447]:
print("3-5: ",len(all_three),'\n3-4: ',len(lvls34),"\n4-5: ",len(lvls45))

3-5:  42 
3-4:  111 
4-5:  154


In [461]:
print("All Three:")
for key, value in lang_dict_all.items():
    print(key,len(value))

All Three:
Korean 22
Chinese 16
Spanish 4


In [462]:
print("3-4:")

for key, value in lang_dict_34.items():
    print(key,len(value))

3-4:
Korean 60
Chinese 42
Spanish 9


In [463]:
print("4-5:")

for key, value in lang_dict_45.items():
    print(key,len(value))

4-5:
Korean 72
Chinese 66
Spanish 16


# Getting distributions of Loose Dataset

In [448]:
# dropping the columns that aren't needed for what we're doing right now
dropped_loose=loose.drop(columns=['3Langs','Item','answer_id','gender','course_id','class_id','question_id','version','text1_len',
                                     'text2_len','text3_len','text1','text2 (line breaks/extra spaces removed, spaces added to reach 60)',
                                     'text3 (edits made to fix word counts)','Judgement','Notes'])
final_loose=dropped_loose.dropna()


In [489]:
# this code gets how many times an id appeared (n_anon_ids) as well as an array of what levels they participated in (lvl_nums)
clean_ids_lvl_loose=final_loose.groupby('anon_id').agg(
    n_anon_ids_loose=('level_id', 'nunique'),
    lvl_nums_loose=('level_id', get_uniques)
)

#changes the clean_ids_lvl's index so we can access the anon_id easier
ind_loose= clean_ids_lvl_loose.reset_index()
ind_loose


,anon_id,n_anon_ids_loose,lvl_nums_loose
0,aa0,1,[5.0]
1,aa3,1,[4.0]
2,aa8,2,"[4.0, 5.0]"
3,aa9,2,"[3.0, 4.0]"
4,ab6,1,[4.0]
...,...,...,...
278,gz2,2,"[4.0, 5.0]"
279,gz5,1,[4.0]
280,gz7,1,[4.0]
281,ha0,1,[5.0]


In [469]:
# for loop that checks if lvl_nums has all three levels in it (3,4, and 5)
# if there are three values in lvl_nums, the anon_id is appended to the all_three list
ind=0
all_three_loose=[]
append=''
for i in ind_loose.anon_id:
    if ind_loose.iat[ind, 1]==3:
        append=ind_loose.at[ind,'anon_id']
        all_three_loose.append(append)
    ind=ind+1

all_three_loose.sort()
len(all_three_loose)


8

In [468]:
# creates list of participants in loose sheet that are in level 3 and 4
ind=0
in34_loose=[]
append=''
for i in ind_loose.anon_id:
    if ind_loose.iat[ind, 1]==2:
        if (3 in ind_loose.iat[ind,2] and 4 in ind_loose.iat[ind,2]):
            append=ind_loose.at[ind,'anon_id']
            in34_loose.append(append)
    elif ind_loose.iat[ind,1]==3:
        append=ind_loose.at[ind,'anon_id']
        in34_loose.append(append)
    ind=ind+1
in34_loose.sort()
print(len(in34_loose))

28


In [467]:
# creates list of participants in loose sheet that are in level 4 and 5
ind=0
in45_loose=[]
append=''
for i in ind_loose.anon_id:
    if ind_loose.iat[ind, 1]==2:
        if (4 in ind_loose.iat[ind,2] and 5 in ind_loose.iat[ind,2]):
            append=ind_loose.at[ind,'anon_id']
            in45_loose.append(append)
    elif ind_loose.iat[ind,1]==3:
        append=ind_loose.at[ind,'anon_id']
        in45_loose.append(append)
    ind=ind+1
in45_loose.sort()
print(len(in45_loose))

61


In [472]:
all_anon_loose=[]
for i in ind_loose.anon_id:
    all_anon_loose.append(i)
all_anon_loose.sort()
len(all_anon_loose)

283

In [512]:
sorted_loose=final_loose.sort_values(by=['anon_id']).reset_index(drop=True)
uloose=sorted_loose['anon_id'].unique()
uloose
#aggfunc={"person":pd.Series.nunique}

array(['aa0', 'aa3', 'aa8', 'aa9', 'ab6', 'ac3', 'ac4', 'ac5', 'ae2',
       'ae4', 'ae9', 'af1', 'af4', 'ag0', 'ag3', 'ag4', 'ag6', 'ag9',
       'ah9', 'aj7', 'aj8', 'al5', 'am5', 'an5', 'an7', 'ao3', 'ao6',
       'ap4', 'ap5', 'aq1', 'aq5', 'ar8', 'ar9', 'as0', 'as4', 'as7',
       'at2', 'at8', 'au3', 'au5', 'au6', 'aw5', 'aw6', 'ax7', 'ay1',
       'az2', 'az8', 'ba3', 'bc0', 'bd7', 'be2', 'be7', 'bf1', 'bf2',
       'bf7', 'bf9', 'bg3', 'bi4', 'bj2', 'bl3', 'bl4', 'bl7', 'bl8',
       'bm0', 'bm4', 'bm9', 'bo8', 'bp4', 'bp9', 'bq0', 'bq8', 'br2',
       'br5', 'br7', 'bs3', 'bt1', 'bt8', 'bu4', 'bu9', 'bv8', 'bv9',
       'bw3', 'bw9', 'bx3', 'by6', 'bz1', 'bz2', 'bz5', 'ca4', 'ca6',
       'ca8', 'ca9', 'cb3', 'cc4', 'cd2', 'cd6', 'ce3', 'cf0', 'cf1',
       'cf6', 'ch0', 'ch2', 'ch9', 'ci0', 'ci4', 'ci9', 'cj5', 'cj8',
       'ck3', 'ck9', 'cl2', 'cm9', 'cn1', 'cn3', 'cn9', 'co5', 'cp0',
       'cq0', 'cq1', 'cs5', 'cs9', 'cu2', 'cu8', 'cv7', 'cw0', 'cw1',
       'cw2', 'cw6',

In [501]:
llang_dict_45={}
for i in all_anon_loose:
    df_ind=binarySearchArr(in45_loose,i)
    key=sorted_loose.at[df_ind,'L1']
    val=sorted_loose.at[df_ind,'anon_id']
    
    if key in llang_dict_45:
        llang_dict_45[key].append(val)
    else:
        llang_dict_45[key]=[val]
    
print(llang_dict_45)

KeyError: -1

In [500]:
print("4-5:")

for key, value in llang_dict_45.items():
    print(key,len(value))

4-5:
Spanish 7
Korean 36
Chinese 18


In [491]:
ind_loose

,anon_id,n_anon_ids_loose,lvl_nums_loose
0,aa0,1,[5.0]
1,aa3,1,[4.0]
2,aa8,2,"[4.0, 5.0]"
3,aa9,2,"[3.0, 4.0]"
4,ab6,1,[4.0]
...,...,...,...
278,gz2,2,"[4.0, 5.0]"
279,gz5,1,[4.0]
280,gz7,1,[4.0]
281,ha0,1,[5.0]


# Answer counts for whole dataset

In [453]:
e_counts=sorted_sac.copy()
e_counts["Number of Questions"]=e_counts.anon_id.count()
entire_counts=e_counts.groupby(['anon_id','native_language'])['Number of Questions'].count()
qcount_entire=entire_counts.reset_index()

In [454]:
qcount_entire

,anon_id,native_language,Number of Questions
0,aa0,Spanish,17
1,aa1,Chinese,22
2,aa3,Chinese,8
3,aa8,Korean,34
4,aa9,Korean,28
...,...,...,...
471,ha0,Korean,3
472,ha2,Korean,44
473,ha6,Korean,32
474,hb4,Korean,39


In [455]:
espanish,echinese,ekorean=find_counts(qcount_entire,all_three,'native_language')
print('Counts in all 3\nSpanish counts: ',espanish,'\nChinese counts: ',echinese,'\nKorean counts: ',ekorean)

Counts in all 3
Spanish counts:  107 
Chinese counts:  468 
Korean counts:  795


In [456]:
espanish34,echinese34,ekorean34=find_counts(qcount_entire,lvls34,'native_language')
print('Counts in 3 and 4\nSpanish counts: ',espanish34,'\nChinese counts: ',echinese34,'\nKorean counts: ',ekorean34)

Counts in 3 and 4
Spanish counts:  156 
Chinese counts:  928 
Korean counts:  1564


In [457]:
espanish45,echinese45,ekorean45=find_counts(qcount_entire,lvls45,'native_language')
print('Counts in 4 and 5\nSpanish counts: ',espanish45,'\nChinese counts: ',echinese45,'\nKorean counts: ',ekorean45)

Counts in 4 and 5
Spanish counts:  298 
Chinese counts:  1712 
Korean counts:  1975


# Answer Counts for Loose dataset

In [143]:
l_counts=final_loose.copy()
l_counts["Number of Questions"]=l_counts.anon_id.count()
loose_counts=l_counts.groupby(['anon_id','L1'])['Number of Questions'].count()
qcount_loose=loose_counts.reset_index()
qcount_loose

,anon_id,L1,Number of Questions
0,aa0,Spanish,8
1,aa3,Chinese,2
2,aa8,Korean,13
3,aa9,Korean,18
4,ab6,Chinese,2
...,...,...,...
278,gz2,Korean,8
279,gz5,Chinese,6
280,gz7,Korean,5
281,ha0,Korean,3


In [163]:
lspanish,lchinese,lkorean=find_counts(qcount_loose, all_three_loose,'L1')
print('Counts in all three\nSpanish counts: ',lspanish,'\nChinese counts: ',lchinese,'\nKorean counts: ',lkorean)

Counts in all three
Spanish counts:  23 
Chinese counts:  34 
Korean counts:  26


In [164]:
lspanish34,lchinese34,lkorean34=find_counts(qcount_loose,in34_loose,'L1')
print('Counts in 3 and 4\nSpanish counts: ',lspanish34,'\nChinese counts: ',lchinese34,'\nKorean counts: ',lkorean34)

Counts in 3 and 4
Spanish counts:  44 
Chinese counts:  64 
Korean counts:  146


In [165]:
lspanish45,lchinese45,lkorean45=find_counts(qcount_loose, in45_loose,'L1')
print('Counts in 4 and 5\nSpanish counts: ',lspanish45,'\nChinese counts: ',lchinese45,'\nKorean counts: ',lkorean45)

Counts in 4 and 5
Spanish counts:  128 
Chinese counts:  305 
Korean counts:  289


In [132]:
espanish45=0
echinese45=0
ekorean45=0
index=0
for i in qcount_entire["Number of Questions"]:
    anon_id=str(qcount_entire.at[index,'anon_id'])
    if anon_id in lvls45:
        if qcount_entire.at[index,'native_language']=='Spanish':            
            espanish45+=i
        elif qcount_entire.at[index,'native_language']=='Chinese':
            echinese45+=i
        elif qcount_entire.at[index,'native_language']=='Korean':
            ekorean45+=i
    index+=1
print('Counts in 4 and 5\nSpanish counts: ',espanish45,'\nChinese counts: ',echinese45,'\nKorean counts: ',ekorean45)

Counts in 4 and 5
Spanish counts:  781 
Chinese counts:  4424 
Korean counts:  4916


# Tokinizing Essays for Loose

In [ ]:
# reference code from previous research


def calculate_tokens(text):
    essay = text

        
    tokenizer = RegexpTokenizer(r'[0-9a-zA-Z_\-]+')
    tokenized = tokenizer.tokenize(essay)
    unique = np.unique(tokenized)
    
    return {
        'total': len(tokenized),
        'unique': len(unique)
    }


directories = ['limited', 'loose', 'strictest']
tokenize_results = { 'filename': [], 'total_tokens': [], 'unique_tokens': [] }

for directory in directories:
    for file in os.listdir(f'./data/in/{directory}'):
        results = calculate_tokens(f'./data/in/{directory}/{file}')
        tokenize_results['filename'].append(file)
        tokenize_results['total_tokens'].append(results['total'])
        tokenize_results['unique_tokens'].append(results['unique'])

pd.DataFrame(data=tokenize_results)

In [226]:
print('Loose All Three')
finding_tokens(loose,all_three_loose,'L1','text3_len')
print(spanish)

Loose All Three
Spanish Token Average:  132.8695652173913 
Korean Token Average:  181.3846153846154 
Chinese Token Average: 188.64705882352942
5379.0


In [214]:
print('Loose 3-4')
finding_tokens(loose,in34_loose,'L1','text3_len')

Loose 3-4
Spanish Token Average:  269.39130434782606 
Korean Token Average:  854.3461538461538 
Chinese Token Average: 319.9117647058824


In [213]:
print('Loose 4-5')
finding_tokens(loose,in45_loose,'L1','text3_len')

Loose 4-5
Spanish Token Average:  1132.5217391304348 
Korean Token Average:  2572.3846153846152 
Chinese Token Average: 2137.8823529411766


In [232]:
print('Loose All Three, Level 3')
finding_tokens_lvl(loose,all_three_loose,'L1','text3_len',3)

Loose All Three, Level 3
Spanish Token Average:  44.26086956521739 
Korean Token Average:  31.03846153846154 
Chinese Token Average: 42.64705882352941


In [233]:
print('Loose All Three, Level 4')
finding_tokens_lvl(loose,all_three_loose,'L1','text3_len',4)

Loose All Three, Level 4
Spanish Token Average:  75.0 
Korean Token Average:  117.84615384615384 
Chinese Token Average: 54.38235294117647


In [234]:
print('Loose All Three, Level 5')
finding_tokens_lvl(loose,all_three_loose,'L1','text3_len',5)

Loose All Three, Level 5
Spanish Token Average:  13.608695652173912 
Korean Token Average:  32.5 
Chinese Token Average: 91.61764705882354


In [235]:
print('Loose 3-4, Level 3')
finding_tokens_lvl(loose,in34_loose,'L1','text3_len',3)

Loose 3-4, Level 3
Spanish Token Average:  94.6086956521739 
Korean Token Average:  261.3076923076923 
Chinese Token Average: 122.05882352941177


In [236]:
print('Loose 3-4, Level 4')
finding_tokens_lvl(loose,in34_loose,'L1','text3_len',4)

Loose 3-4, Level 4
Spanish Token Average:  161.17391304347825 
Korean Token Average:  560.5384615384615 
Chinese Token Average: 106.23529411764706


In [237]:
print('Loose 4-5, Level 4')
finding_tokens_lvl(loose,in45_loose,'L1','text3_len',4)

Loose 4-5, Level 4
Spanish Token Average:  616.304347826087 
Korean Token Average:  1285.9615384615386 
Chinese Token Average: 931.1764705882352


In [238]:
print('Loose 4-5, Level 5')
finding_tokens_lvl(loose,in45_loose,'L1','text3_len',5)

Loose 4-5, Level 5
Spanish Token Average:  471.95652173913044 
Korean Token Average:  1255.3846153846155 
Chinese Token Average: 1164.0588235294117


# Tokenizing Essays for Whole Dataset

In [212]:
print('Whole All Three')
finding_tokens(stu_ans_crs,all_three,'native_language','text_len')

Whole All Three
Spanish Token Average:  991.7391304347826 
Korean Token Average:  7537.307692307692 
Chinese Token Average: 3345.9117647058824


In [216]:
print('Whole 3-4')
finding_tokens(stu_ans_crs,lvls34,'native_language','text_len')

Whole 3-4
Spanish Token Average:  1574.2608695652175 
Korean Token Average:  12814.961538461539 
Chinese Token Average: 6310.088235294118


In [217]:
print('Whole 4-5')
finding_tokens(stu_ans_crs,lvls45,'native_language','text_len')

Whole 4-5
Spanish Token Average:  3162.478260869565 
Korean Token Average:  19977.19230769231 
Chinese Token Average: 13701.588235294117


In [240]:
print('Whole All Three, Level 3')
finding_tokens_lvl(stu_ans_crs,all_three,'native_language','text_len',3)

Whole All Three, Level 3
Spanish Token Average:  234.2608695652174 
Korean Token Average:  1290.6538461538462 
Chinese Token Average: 567.4117647058823


In [241]:
print('Whole All Three, Level 4')
finding_tokens_lvl(stu_ans_crs,all_three,'native_language','text_len',4)

Whole All Three, Level 4
Spanish Token Average:  591.2173913043479 
Korean Token Average:  3502.1153846153848 
Chinese Token Average: 1308.6470588235295


In [242]:
print('Whole All Three, Level 5')
finding_tokens_lvl(stu_ans_crs,all_three,'native_language','text_len',5)

Whole All Three, Level 5
Spanish Token Average:  166.2608695652174 
Korean Token Average:  2735.769230769231 
Chinese Token Average: 1449.2941176470588


In [243]:
print('Whole 3-4, Level 3')
finding_tokens_lvl(stu_ans_crs,lvls34,'native_language','text_len',3)

Whole 3-4, Level 3
Spanish Token Average:  431.2173913043478 
Korean Token Average:  3155.8846153846152 
Chinese Token Average: 1775.5588235294117


In [244]:
print('Whole 3-4, Level 4')
finding_tokens_lvl(stu_ans_crs,lvls34,'native_language','text_len',4)

Whole 3-4, Level 4
Spanish Token Average:  976.7826086956521 
Korean Token Average:  6687.346153846154 
Chinese Token Average: 3064.676470588235


In [245]:
print('Whole 4-5, Level 4')
finding_tokens_lvl(stu_ans_crs,lvls45,'native_language','text_len',4)

Whole 3-4, Level 4
Spanish Token Average:  1584.695652173913 
Korean Token Average:  9307.923076923076 
Chinese Token Average: 5546.235294117647


In [246]:
print('Whole 4-5, Level 4')
finding_tokens_lvl(stu_ans_crs,lvls45,'native_language','text_len',5)

Whole 3-4, Level 4
Spanish Token Average:  1343.5217391304348 
Korean Token Average:  9369.846153846154 
Chinese Token Average: 7567.382352941177


In [219]:
loose

,3Langs,Item,answer_id,anon_id,L1,gender,course_id,level_id,class_id,question_id,version,text1_len,text2_len,text3_len,text1,"text2 (line breaks/extra spaces removed, spaces added to reach 60)",text3 (edits made to fix word counts),Judgement,Notes
0,Y,C12,141.0,aj8,Chinese,Male,118.0,5.0,w,17.0,1.0,121.0,114.0,114.0,"\n\n\n Today, I am going to describe one of my...","Today, I am going to describe one of my classm...","Today, I am going to describe one of my classm...",1.0,NaN
1,Y,C13,143.0,az8,Chinese,Female,118.0,5.0,w,17.0,1.0,96.0,95.0,95.0,My niece is 3 years old who is my younger brot...,My niece is 3 years old who is my younger brot...,My niece is 3 years old who is my younger brot...,1.0,NaN
2,Y,K9,133.0,az2,Korean,Male,118.0,5.0,w,17.0,1.0,130.0,128.0,128.0,"When I was in Germany, I met a friend who was ...","When I was in Germany, I met a friend who was ...","When I was in Germany, I met a friend who was ...",1.0,NaN
3,Y,K11,135.0,at8,Korean,Female,118.0,5.0,w,17.0,3.0,104.0,105.0,105.0,my friend is ANON_NAME_0. she is a my ELI frie...,my friend is ANON_NAME_0. she is a my ELI frie...,my friend is ANON_NAME_0. she is a my ELI frie...,1.0,NaN
4,Y,K20,188.0,eh9,Korean,Male,118.0,5.0,w,17.0,1.0,98.0,97.0,97.0,Younghun is my best friend. His facial appeara...,Younghun is my best friend. His facial appeara...,Younghun is my best friend. His facial appeara...,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1807,Y,C3207,48246.0,bv8,Chinese,Unknown,1043.0,4.0,g,6105.0,1.0,133.0,131.0,131.0,The quote that talks about a true friend is so...,The quote that talks about a true friend is so...,The quote that talks about a true friend is so...,3.0,NaN
1808,Y,K3559,48254.0,dr8,Korean,Female,1043.0,4.0,g,6105.0,1.0,175.0,162.0,162.0,I think this means that home is not just a pla...,I think this means that home is not just a pla...,I think this means that home is not just a pla...,3.0,NaN
1809,Y,S781,48252.0,bl3,Spanish,Female,1043.0,4.0,g,6105.0,1.0,157.0,146.0,146.0,This quotation means that the majority of peop...,This quotation means that the majority of peop...,This quotation means that the majority of peop...,3.0,NaN
1810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [250]:
test=stu_ans_crs.drop(columns=['course_id','age','version','question_id','gender'])
test

,anon_id,native_language,answer_id,text_len,text,tokens,level_id
0,do6,Russian,150,299,Some people prefer eat out and some like doing...,"['Some', 'people', 'prefer', 'eat', 'out', 'an...",5
1,do6,Russian,1221,288,My opinion is that a person does need educatio...,"['My', 'opinion', 'is', 'that', 'a', 'person',...",5
2,do6,Russian,1957,321,There are two national rooms in the Cathedral ...,"['There', 'are', 'two', 'national', 'rooms', '...",5
3,do6,Russian,2164,464,There are two nation rooms in the Cathedral of...,"['There', 'are', 'two', 'nation', 'rooms', 'in...",5
4,bv5,Arabic,151,315,"""Not all learning takes place in the classroom...","['``', 'Not', 'all', 'learning', 'takes', 'pla...",5
...,...,...,...,...,...,...,...
46225,cy7,Korean,47682,10,1. emphasis\n2. appropriate\n3. requires\n4. a...,"['1', '.', 'emphasis', '2', '.', 'appropriate'...",3
46226,fp7,Turkish,47823,10,1.emphasis \n2.appropriate\n3.requires\n4.anal...,"['1', '.', 'emphasis', '2', '.', 'appropriate'...",3
46227,fq6,Chinese,47824,10,1.emphasis\n2.appropriate\n3.requires\n4.analy...,"['1', '.', 'emphasis', '2', '.', 'appropriate'...",3
46228,di6,Chinese,47787,10,1 emphasis\n2 normal\n3 requires\n4 analyze\n5...,"['1', 'emphasis', '2', 'normal', '3', 'require...",3


1- emphasis
2- appropriate
3- requires
4- analyze
5- authority
6- significant
7- flexible
8- challenge
9- normal
10- contact
